In [1]:
import io.github.cdimascio.dotenv.Dotenv
import java.nio.file.Paths

val dotenv = Dotenv.load()
val dataDir = dotenv.get("DATA_DIR").let { Paths.get(it).toFile() }.also { it.mkdirs() }
dataDir

/home/mm/w/jpatch/data

In [2]:
import kotlinx.serialization.Serializable

// from previous notebook
@Serializable
data class InterimResult2(
    val cveId: String,
    val ghsaId: String,
    val packageName: String,
    val vulnIntroduced: String,
    val latestVulnVersion: String,
    val fixedVersion: String,
    val fixCommitRepo: String,
    val fixCommitHash: String,
    val jarBefore: String,
    val jarAfter: String,
    val srcBefore: String,
    val srcAfter: String,
    val classFileBefore: String,
    val classFileAfter: String,
    val changedClass: String,
    val changedMethods: String,
    val packageIdBefore: String,
    val packageIdAfter: String,
    val diffParsedJson: String,
    val nloc: Int,
)

In [3]:
val data = DataFrame.readJson(dataDir.resolve("proc/changes.json"))

In [5]:
import com.google.gson.GsonBuilder
import com.google.gson.JsonElement

data.rows().shuffled().take(1).map {
    val jsonString = it.diffParsedJson
    DISPLAY(jsonString)
    val gson = GsonBuilder().setPrettyPrinting().create()
    val json = gson.fromJson(jsonString, JsonElement::class.java)
    val niceJsonString = gson.toJson(json)
    DISPLAY(niceJsonString)
}

{"added": [[226, "     * @param pStrictChecking whether to a strict check (i.e. server side check)"], [227, "     * @return true if if cors access is allowed"], [229, "    public boolean isOriginAllowed(String pOrigin,boolean pStrictChecking) {"], [230, "        return restrictor.isOriginAllowed(pOrigin, pStrictChecking);"]], "deleted": [[226, "     * @return true if icors access is allowed"], [228, "    public boolean isCorsAccessAllowed(String pOrigin) {"], [229, "        return restrictor.isCorsAccessAllowed(pOrigin);"]]}

{
  "added": [
    [
      226,
      "     * @param pStrictChecking whether to a strict check (i.e. server side check)"
    ],
    [
      227,
      "     * @return true if if cors access is allowed"
    ],
    [
      229,
      "    public boolean isOriginAllowed(String pOrigin,boolean pStrictChecking) {"
    ],
    [
      230,
      "        return restrictor.isOriginAllowed(pOrigin, pStrictChecking);"
    ]
  ],
  "deleted": [
    [
      226,
      "     * @return true if icors access is allowed"
    ],
    [
      228,
      "    public boolean isCorsAccessAllowed(String pOrigin) {"
    ],
    [
      229,
      "        return restrictor.isCorsAccessAllowed(pOrigin);"
    ]
  ]
}

[kotlin.Unit]

In [25]:
import com.google.gson.JsonArray

data.rows().shuffled().take(1).map {
    val stringArray = it.changedMethods
    DISPLAY(stringArray)
    val elems = "[" + stringArray.trim().trim('[', ']').split(',').map { "\"${it.trim()}\"" }.joinToString(", ") + "]"
    DISPLAY(elems)
    val gson = GsonBuilder().setPrettyPrinting().create()
    val json = gson.fromJson(elems, JsonArray::class.java)
    val niceJsonString = gson.toJson(json)
    DISPLAY(niceJsonString)
}

[C3P0ConfigXmlUtils::extractXmlConfigFromInputStream, C3P0ConfigXmlUtils::extractXmlConfigFromDefaultResource, C3P0ConfigXmlUtils::extractXmlConfigFromDefaultResource, C3P0ConfigXmlUtils::attemptSetFeature, C3P0ConfigXmlUtils::extractXmlConfigFromInputStream, C3P0ConfigXmlUtils::cautionDocumentBuilderFactory]

["C3P0ConfigXmlUtils::extractXmlConfigFromInputStream", "C3P0ConfigXmlUtils::extractXmlConfigFromDefaultResource", "C3P0ConfigXmlUtils::extractXmlConfigFromDefaultResource", "C3P0ConfigXmlUtils::attemptSetFeature", "C3P0ConfigXmlUtils::extractXmlConfigFromInputStream", "C3P0ConfigXmlUtils::cautionDocumentBuilderFactory"]

[
  "C3P0ConfigXmlUtils::extractXmlConfigFromInputStream",
  "C3P0ConfigXmlUtils::extractXmlConfigFromDefaultResource",
  "C3P0ConfigXmlUtils::extractXmlConfigFromDefaultResource",
  "C3P0ConfigXmlUtils::attemptSetFeature",
  "C3P0ConfigXmlUtils::extractXmlConfigFromInputStream",
  "C3P0ConfigXmlUtils::cautionDocumentBuilderFactory"
]

[kotlin.Unit]

In [26]:
import com.google.gson.JsonArray

data class InterimResult3(
    val cveId: String,
    val ghsaId: String,
    val groupArtifact: String,
    val vulnIntroduced: String,
    val latestVulnVersion: String,
    val fixedVersion: String,
    val fixCommitRepo: String,
    val fixCommitHash: String,
    val jarBefore: String,
    val jarAfter: String,
    val srcBefore: String,
    val srcAfter: String,
    val classFileBefore: String,
    val classFileAfter: String,
    val changedClass: String,
    val changedMethods: JsonArray,
    val packageIdBefore: String,
    val packageIdAfter: String,
    val diffJson: JsonElement,
    val nloc: Int,
)

val newData = data
    .rows()
    .map {
        val gson = GsonBuilder().setPrettyPrinting().create()
        val diffParsedJson = gson.fromJson(it.diffParsedJson, JsonElement::class.java)
        val changedMethodsJson =
            gson.fromJson("[" + it.changedMethods.trim().trim('[', ']').split(',').map { "\"${it.trim()}\"" }
                .joinToString(", ") + "]", JsonArray::class.java)
        
        InterimResult3(
            cveId = it.cveId,
            ghsaId = it.ghsaId,
            groupArtifact = it.packageName,
            vulnIntroduced = it.vulnIntroduced,
            latestVulnVersion = it.latestVulnVersion,
            fixedVersion = it.fixedVersion,
            fixCommitRepo = it.fixCommitRepo,
            fixCommitHash = it.fixCommitHash,
            jarBefore = it.jarBefore,
            jarAfter = it.jarAfter,
            srcBefore = it.srcBefore,
            srcAfter = it.srcAfter,
            classFileBefore = it.classFileBefore,
            classFileAfter = it.classFileAfter,
            changedClass = it.changedClass,
            changedMethods = changedMethodsJson,
            packageIdBefore = it.packageIdBefore,
            packageIdAfter = it.packageIdAfter,
            diffJson = diffParsedJson,
            nloc = it.nloc,
        )
    }

In [27]:
import io.ktor.util.*
import java.io.FileOutputStream
import java.io.OutputStreamWriter
import java.nio.charset.Charset

val gson = GsonBuilder().setPrettyPrinting().disableHtmlEscaping().create() 
val json = gson.toJson(newData)
dataDir.resolve("proc/changes.v2.json").writeText(json)